# 1. kodutöö - klassifitseerimine ja regressioon tõenäosusliku modelleerimise abil

Importe võite lisada, ent kõikvõimalikud moodulid, mida pole seni praktikumi materjalides kasutatud ja mis vajaksid lisainstalli, tuleb Moodle'i tekstiväljas eraldi raporteerida ja piisavalt (väga põhjalikult ja veenvalt) põhjendada.

In [ ]:
import sklearn
import pandas as pd
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
#Failide lugemiseks Colabis
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/ai")

Mounted at /content/drive


In [ ]:
faili_asukoht = "imdb.csv"

## 1. Loe andmed failist sisse

All on väike kontroll selle kohta, kas andmed said sisse loetud samal kujul.

In [ ]:
# Loe sisse kõik treeningandmed pandas DataFrame'iks muutujasse filmid
filmid = pd.read_csv(faili_asukoht, delimiter=",",index_col=0)
# Kontrolli, kas andmed on sobival kujul
assert filmid.shape == (4538, 28)

## 2. Töötle andmeid

**2 punkti** saab andmete eeltöötluse eest. See tähendab, et tuleb andmetele otsa vaadata, sobivad välja valida, vajadusel nende väärtusi või formaati muuta jne. Soovi korral võid siin teha väljatrükke, visualiseerimisi jms.  

Korrektse andmete eeltöötluse tulemus on muuhulgas see, et mõlemad ennustusmeetodid (*predictScore* ja *predictRating*, vt allpool) töötavad. Soovi korral võib siin teha ka nt mitu eraldi eeltöötlust, vastavalt edaspidi kasutatavale mudelile.  

**NB!** Tagastatavaid struktuure ja väärtusi võite ise valida ja muuta, oluline on see, et treening- ja testfunktsioon nendega toime tuleksid. Vaadake üle, et töölehe puhtal läbijooksutamisel midagi kaotsi ei läheks.

In [ ]:
from sklearn.impute import SimpleImputer

def preprocess(dataFrame):
    '''Tegeleb sisse saadud DataFrame'i töötlemisega. 
    '''
    # võimalikud testandmete eeltöötluseks vajalikud mudelid-teadmised (nt keskmised väärtused vms)
    andmestikud = {
        'regression_X': ['num_critic_for_reviews', 'duration', 'num_voted_users', 'num_user_for_reviews', 'Drama'],
        'bad_words': ['drug', 'abuse', 'crime', 'sex', 'rape', 'murder', 'death', 'massacre'],
        'good_words': ['family', 'fun', 'children', 'adventure']
    }
    # eeltöötluse tulemus - muudetud DataFrame
    processedDataFrame = dataFrame
  
    # eemaldame read, kus mõne vajaliku tulba andmed puuduvad
    processedDataFrame = processedDataFrame.dropna(subset=['content_rating', 'num_critic_for_reviews', 'duration', 'num_user_for_reviews'])

    # tekitame žanritest one-hotid
    andmestikud['classifier_X'] = set([item for sublist in processedDataFrame['genres'].str.split('|').values for item in sublist])
    for genre in andmestikud['classifier_X']:
        processedDataFrame[genre] = processedDataFrame.apply(
            lambda row : 1 if genre in row['genres'] else 0, axis=1
        )

    # sobitame imputerid tulpadele, mille väärtused testreal kindlasti olema peavad (nende põhjal ennustatakse)
    andmestikud['num_critic_for_reviews_imp'] = SimpleImputer(strategy='mean').fit(processedDataFrame[['num_critic_for_reviews']])
    andmestikud['duration_imp'] = SimpleImputer(strategy='median').fit(processedDataFrame[['duration']])
    andmestikud['num_voted_users_imp'] = SimpleImputer(strategy='mean').fit(processedDataFrame[['num_voted_users']])
    andmestikud['num_user_for_reviews_imp'] = SimpleImputer(strategy='mean').fit(processedDataFrame[['num_user_for_reviews']])

    return processedDataFrame, andmestikud

## 3. Filmireitingute ennustamine  

**2 punkti** saab mudeli eest, mis ennustab filmi reitingu (tabelis tulp *'content_rating'*). Tegu on klassifitseerimisülesandega

### 3.1. Treeni mudel

In [ ]:
from sklearn.svm import SVC

def trainRating(filmiDataFrame):
    '''Õpib sisendandmete põhjal mudeli, mis oskab ennustada filmi reitingu.
       Võib kasutada väliseid funktsioone ja muud endale sobivat.
       Sisendiks saab treeningandmete DataFrame'i.
       Tagastab treenitud mudeli.'''

    treeningandmed, andmestikud = preprocess(filmiDataFrame)
    
    X = treeningandmed[andmestikud['classifier_X']]
    y = treeningandmed['content_rating']

    mudel = SVC(gamma=2, C=1)
    mudel.fit(X, y)

    return mudel, andmestikud

### 3.2. Ennustamine

Meeldetuletus: andmete eeltöötlus tuleb teha ka testhulgale. Selleks võib kasutada abifunktsioone. Siia võib lisada abiplokke.

In [ ]:
def predictRating(mudel, andmestikud, testhulk):
    '''Saab sisendiks treenitud mudeli, eeltöötluseks vajalikud andmed 
       ja terve testhulga algsel kujul (ilma reitinguta ja eeltöötluseta).
       Tagastab ennustatud reitingute järjendi.'''
    
    for genre in andmestikud['classifier_X']:
        testhulk[genre] = testhulk.apply(
            lambda row: 1 if genre in row['genres'] else 0, axis=1
        )

    return mudel.predict(testhulk[andmestikud['classifier_X']])

### 3.3. Testimine  

Siin on etteantud testplokk, mis näitab, kuidas see kõik koos toimida võiks. Oluline on see, et see kõik oleks kasutatav erinevatel andmehulkadel, näiteks varem nägemata failil.  Jah, muidugi võib seda töö käigus muuta - kontrollimisel eeldatakse aga, et see algne testplokk ka töötab.

Üldiselt on alguses mõttekas väiksema andmehulgaga katsetada, kui süntaksi- ja muud vead on välja roogitud, võib kogu andmehulga ette võtta.  

**NB!** Ärge eeldage, et *testandmed* väärtustamisel saab midagi juba töödeldut. Testandmete eeltöötlus tuleks ära teha ennustusmeetodi raames või mõne abifunktsiooniga enne seda. Tavaelus tulevad testandmed teile uue ja iseseisvana ette.

In [ ]:
# Eeldades, et 'filmid' on äsja failist sisseloetud, 28-tulbaline ja muidu muutmata kujul DataFrame
mudel, andmestikud = trainRating(filmid[100:])

# Võta testimiseks esimesed 10 kirjet, ilma ennustatava väärtuseta
# Kodutöö hindamisel loetakse testandmed eraldi failist, treenimiseks kasutatakse kogu treeninghulka
testandmed = filmid[:100]

# Tunnused, mille põhjal ennustada
testandmed_x = testandmed.drop('content_rating', axis = 1)
# Tunnus, mida ennustada
testandmed_y = testandmed['content_rating']

# Ennustatud reitingud
ennustused = []
ennustused = predictRating(mudel,andmestikud,testandmed_x)

# Arvuta ennustuste ja õigete vastuste põhjal ennustamise täpsus
print("Täpsus", accuracy_score(ennustused, testandmed_y.astype(str)))
# Täpsus on väiksem, sest antud testhulgas on ka filme, mis ei ole kas R või PG

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Täpsus 0.55


## 4. Filmiskooride ennustamine  

**2 punkti** saab mudeli eest, mis ennustab filmi IMDB skoori. Kuna see on pidev väärtus, siis kasutame regressiooni ja mõõdikuna keskmist ruutviga (MSE).

### 4.1. Treeni mudel


In [ ]:
from sklearn.linear_model import LinearRegression

def trainScore(filmiDataFrame):
    '''Õpib sisendandmete põhjal mudeli, mis oskab ennustada IMDB skoori.
       Võib kasutada väliseid funktsioone ja muud endale sobivat.
       Sisendiks saab treeningandmete DataFrame'i.
       Tagastab treenitud mudeli.'''

    treeningandmed, andmestikud = preprocess(filmiDataFrame)

    X = treeningandmed[andmestikud['regression_X']]
    y = treeningandmed['imdb_score']

    mudel = LinearRegression()
    mudel.fit(X, y)

    return mudel, andmestikud

### 4.2. Ennustamine

Meeldetuletus: andmete eeltöötlus tuleb teha ka testhulgale. Selleks võib kasutada abifunktsioone. Siia võib lisada abiplokke.

In [ ]:
def predictScore(mudel,andmestikud, testhulk):
    '''Saab sisendiks treenitud mudeli, eeltöötluseks vajalikud andmed 
       ja terve testhulga algsel kujul (ilma skoorita ja eeltöötluseta).
       Tagastab ennustatud skooride järjendi.'''

    testhulk['num_critic_for_reviews'] = andmestikud['num_critic_for_reviews_imp'].transform(testhulk[['num_critic_for_reviews']])
    testhulk['duration'] = andmestikud['duration_imp'].transform(testhulk[['duration']])
    testhulk['num_voted_users'] = andmestikud['num_voted_users_imp'].transform(testhulk[['num_voted_users']])
    testhulk['num_user_for_reviews'] = andmestikud['num_user_for_reviews_imp'].transform(testhulk[['num_user_for_reviews']])

    # tuleb välja, et draamafilmidel on keskmisest kõrgem reiting :)
    testhulk['Drama'] = testhulk.apply(
        lambda row: 1 if 'Drama' in row['genres'] else 0, axis=1
    )
      
    return mudel.predict(testhulk[andmestikud['regression_X']])

### 4.3. Testimine

Siin on etteantud testplokk, mis näitab, kuidas see kõik koos toimida võiks. Oluline on see, et see kõik oleks kasutatav erinevatel andmehulkadel, näiteks varem nägemata failil. Jah, muidugi võib seda töö käigus muuta - kontrollimisel eeldatakse aga, et see algne testplokk ka töötab.

Üldiselt on alguses mõttekas väiksema andmehulgaga katsetada, kui süntaksi- ja muud vead on välja roogitud, võib kogu andmehulga ette võtta.  

**NB!** Ärge eeldage, et *testandmed* väärtustamisel saab midagi juba töödeldut. Testandmete eeltöötlus tuleks ära teha ennustusmeetodi raames või mõne abifunktsiooniga enne seda. Tavaelus tulevad testandmed teile uue ja iseseisvana ette.

In [ ]:
# Eeldades, et 'filmid' on äsja failist sisseloetud, 28-tulbaline ja muidu muutmata kujul DataFrame
mudel, andmestikud = trainScore(filmid[200:])

# Võta testimiseks esimesed 10 kirjet, ilma ennustatava väärtuseta
# Kodutöö hindamisel loetakse testandmed eraldi failist, treenimiseks kasutatakse kogu treeninghulka
testandmed = filmid[:200]

# Tunnused, mille põhjal ennustada
testandmed_x = testandmed.drop('imdb_score', axis = 1)
# Tunnus, mida ennustada
testandmed_y = testandmed['imdb_score']

# Ennustatud filmiskoorid
ennustused = predictScore(mudel,andmestikud,testandmed_x)

# Arvuta ennustuste ja õigete vastuste põhjal MSE
print("MSE:", mean_squared_error(ennustused, testandmed_y))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


MSE: 0.9651257048871282


## 5. Mudeli kirjeldus

**1 punkti** saab enda mudelite (sh andmete valiku ja eeltöötlusotsuste) kirjeldamise eest. Mis sorti eeltöötluse tegid ja miks? Mille alusel valisid lõpliku mudeli?

Kuna andmeid oli piisavalt, siis treenimise jaoks viskasin minema read, kus mõni ennustamiseks vajalik parameeter puudus. Samas aga tegin ülejäänutest sama tulba andmetest Imputeri (olenevalt suuruse varieeruvusest kas mediaan- või aritmeerilise keskmise põhjal) selleks, et vajadusel asendada testhulgal puuduvaid andmeid. Kuna žanrid on olulisel kohal määramaks, kas film sobib lastele või mitte, teen treeninghulga andmete žanritest "one-hotid" (vahega, et ühel filmil võib tegelikult olla mitu žanrit) ning kasutan kõiki neid tegelikult ka ennustamisel (ehk ennustamiseks vajalikud parameetrid sõltuvad treeningandmetest). 

Klassifitseerimiseks kasutasin SVC-d ning skooride ennustamiseks LinearRegressionit, sest need töötavad hästi :) SVC 2 gammaga andis oluliselt parema tulemuse kui näiteks KNeighbors.

## 6. Tagasiside

Siia võib järgi anda tagasiside kodutööle. Tagasiside on vabatahtlik ja sellega aitate maailma (täpsemalt tehisintellekti ainet) paremaks muuta.  
Kas 1. kodutöö oli kohutavalt lihtne, meeletult keeruline, teistmoodi huvitav või lihtsalt igav? Kas miski tekitas segadust või oli liiga detailselt ette antud? Mis osa kodutööst oli kõige raskem? Mitu tundi see umbes aega võttis ja kas midagi tuleks järgmisel korral kindlasti teisiti teha? 

Lahe kodutöö, aga võibolla natuke ebaselgelt väljendatud, mida soovitakse. Tegin ka alguses klassifitseerimine PG või R loogikaga, aga õnneks enne esitamist nägin, et tegelt tuleb kõiki reitinguid hinnata. Raskuselt oli tip-top.